## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [1]:
!pip show requests

Name: requests
Version: 2.32.2
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: C:\Users\vega2\anaconda3\Lib\site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: anaconda-catalogs, anaconda-client, anaconda-cloud-auth, anaconda-project, conda, conda-build, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, intake, jupyterlab_server, panel, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract


In [2]:
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: 
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: C:\Users\vega2\anaconda3\Lib\site-packages
Requires: soupsieve
Required-by: conda-build, nbconvert


In [1]:
import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
print(f'request ver {requests.__version__}' )
print('beautifulsoup {}'.format(bs4.__version__))

request ver 2.32.2
beautifulsoup 4.12.3


### 1. 뉴스 제목 추출하기

In [3]:
import requests
import bs4
from bs4 import BeautifulSoup


# IT/과학 뉴스
req_param = {
    'sid': 105
}
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)
# 요청 헤더 설정 : 브라우저 정보
req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}
res = requests.get(url, headers=req_header)
print(type(res))
print(res.status_code)
if res.ok:
    html = res.text #소스보기 텍스트
    soup = BeautifulSoup(html, 'html.parser')
    # <div class="sa_text">
    # atag_list = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article']")
    atag_list = soup.select("div.sa_text a.sa_text_title")
    print(len(atag_list))
    print(type(atag_list))
    for atag in atag_list:
        # print(atag)
        title = atag.text
        link = atag['href']
        # print(f'{title}')
        # print(f'기사링크 = {link}')

https://news.naver.com/section/105
<class 'requests.models.Response'>
200
44
<class 'bs4.element.ResultSet'>


In [6]:
section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}

### 1.1 뉴스제목 추출하는 함수 선언하기

In [4]:
import requests
from bs4 import BeautifulSoup

def print_news(sid, section):    
    # 요청 Parameter
    req_param = {
        'sid': sid
    }
    url = 'https://news.naver.com/section/{sid}'.format(**req_param)
    
    print(f'======> {url} {section} 뉴스 <======')
    
    # 요청 헤더 설정 : 브라우저 정보
    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }

    res = requests.get(url, headers=req_header)   
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        # <div class="sa_text">
        # atag_list = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article']")
        atag_list = soup.select("div.sa_text a.sa_text_title")
        print(len(atag_list))
        
        for atag in atag_list:
            title = atag.text
            link = atag['href']
            # print(f'{title}')
            # print(f'기사링크 = {link}')
    else:
        print(f'Error Code = {res.status_code}') 

In [7]:
print_news(104,section_dict[104])

======> https://news.naver.com/section/104 세계 뉴스 <======
46


### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [8]:
import requests
import os

req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=798173&no=5&amp;weekday=thu'
}
img_urls = [
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg'
]

for img_url in img_urls:
    res = requests.get(img_url, headers=req_header)
    if res.ok:        
        img_data = res.content
        file_name = os.path.basename(img_url)
        # binday data를 file에 write하기
        with open(file_name,'wb') as file:
            print(f'writing to {file_name}({len(img_data):,} bytes)')
            file.write(img_data)
    else:
        print(f'Error Code = {res.status_code}')         

writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_1.jpg(299,319 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_2.jpg(163,333 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(182,530 bytes)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [9]:
import requests
import os

url = 'https://comic.naver.com/webtoon/detail?titleId=817945&no=37&week=mon'
req_header = {
    'referer': url
}
res = requests.get(url)
if res.ok: #200
    # xx.jpg 파일명 목록 가져오기
    soup = BeautifulSoup(res.text, 'html.parser')
    img_tags = soup.select("img[src*='IMAG01']")
    print(len(img_tags))
    
    img_url_list = list()
    for img_tag in img_tags:
        img_url_list.append(img_tag['src'])

    imgdir_name = 'img'
    # img 디렉토리가 없으면 생성
    if not os.path.isdir(imgdir_name):
        os.mkdir(imgdir_name)

    for img_url in img_url_list:
        res = requests.get(img_url, headers=req_header)
        if res.ok:        
            img_data = res.content
            dir_name = os.path.join(imgdir_name, os.path.basename(img_url))
            # binday data를 file에 write하기
            with open(dir_name,'wb') as file:
                print(f'writing to {file_name}({len(img_data):,} bytes)')
                file.write(img_data)
        else:
            print(f'Error Code = {res.status_code}')         
else:
    print(res.status_code)

18
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(128,897 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(143,194 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(101,900 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(112,007 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(190,876 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(134,590 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(132,910 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(174,434 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(88,576 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(177,240 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(95,130 bytes)
writing to 20220804112251_d97bd1e1b38f0cd0

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [69]:
import requests

upload_files = {
    'img1': open('img/f1.jpg','rb'),
    'img2': open('img/f2.jpg','rb'),
}
url = 'http://httpbin.org/post'
res = requests.post(url, files=upload_files)
print(res.status_code)
#print(res.json()['files']['img1'])


200


### 4. 단축 URL 생성하는 Open API 사용하기
* urllib 사용

In [25]:
# 캡차 키 발급 요청 
import os
import sys
import urllib.request

client_id = "_Jc5Vu9T9Sx8QJs_y6Mk" # 개발자센터에서 발급받은 Client ID 값
client_secret = "_wJEELYvtJ" # 개발자센터에서 발급받은 Client Secret 값

code = "0"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"key":"NhXvIk2vw6BCpezP"}


In [17]:
# 캡차 이미지 요청
import os
import sys
import urllib.request

client_id = "_J" # 개발자센터에서 발급받은 Client ID 값
client_secret = "_w" # 개발자센터에서 발급받은 Client Secret 값
key = "N" # 캡차 Key 값

url = "https://openapi.naver.com/v1/captcha/ncaptcha.bin?key=" + key
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode==200):
    print("캡차 이미지 저장")
    response_body = response.read()
    with open('captcha.jpg', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

캡차 이미지 저장


In [24]:
# 사용자 입력값 검증 요청
import os
import sys
import urllib.request

client_id = "_Jc" # 개발자센터에서 발급받은 Client ID 값
client_secret = "_w" # 개발자센터에서 발급받은 Client Secret 값
key = "p" # 캡차 Key 값

code = "1"
value = "D6P5SXY7"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code + "&key=" + key + "&value=" + value
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"result":true,"responseTime":141.65}


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [27]:
import requests

client_id = "_Jc" # 개발자센터에서 발급받은 Client ID 값
client_secret = "_w" # 개발자센터에서 발급받은 Client Secret 값
key = "N" # 캡차 Key 값

code = "1"
value = "Y452"

url = "https://openapi.naver.com/v1/captcha/nkey"
params = {
    "code": code,
    "key": key,
    "value": value
}

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

response = requests.get(url, params=params, headers=headers)
print(response)

if response.status_code == 200:
    print(response.text)  # 응답 본문 출력
else:
    print(f"Error Code: {response.status_code}")

<Response [200]>
{"result":true,"responseTime":56.84}


### 5. 블로그 검색하기

In [31]:
import requests
import pprint

headers = {
    'X-Naver-Client-Id': '_Jc5Vu9T9Sx8QJs_y6Mk',
    'X-Naver-Client-Secret': '_wJEELYvtJ',
}

payload = {
    'query': 'Llama 3',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'

res = requests.get(url, params=payload, headers=headers)
if response.status_code == 200:
    items_data = res.json()['items']
    print(items_data)
    # [{},{},{}]
    
    items_list = list()
    item_list = []
    for item in items_data:
    #     print(item)
        item_list.append(item['title'])
        item_list.append(item['bloggername'])
        item_list.append(item['description'])
        item_list.append(item['bloggerlink'])
        item_list.append(item['link'])
    
        items_list.append(item_list)
        item_list = []
    
    print(items_list) #[[],[]]
    
    with open('data/nhnblog.txt','w',encoding="utf-8")as file:
        for items in items_list:
            for item in items:
                item = item + '\n'
                file.write(item)
            file.write('-'*150+'\n')
else:
    print(f"Error Code: {response.status_code}")            

{'lastBuildDate': 'Thu, 06 Feb 2025 00:10:42 +0900', 'total': 11781, 'start': 1, 'display': 100, 'items': [{'title': '[AI] Meta <b>Llama 3</b> 설치 및 Open WebUI 실행까지 (Windows)', 'link': 'https://megastorage.tistory.com/507', 'description': "Open WebUI 정상 동작 확인 다시 메인페이지로 돌아온 후 상단의 'Select a model' 선택 후 '<b>llama3</b>:8b'를 선택합니다. 이제 <b>llama 3</b> 8B 언어 모델과 대화를 할 수 있게 되었습니다. 테스트 대화를... ", 'bloggername': '삼런이의 만물상', 'bloggerlink': 'https://megastorage.tistory.com/', 'postdate': '20240428'}, {'title': '라마3 설치방법, 정말 쉽다!<b>LLAMA3</b> + Ollama+ 동영상 포함', 'link': 'https://blog.naver.com/ocmarketing/223424810886', 'description': '라마3, <b>LLAMA3</b>가 무엇인가? 여기에 대해서는 이전에 포스팅을 참고해 주세요. 이 글에서는 라마3을... 이중에서 우리의 관심사인 라마3 <b>LLAMA3</b>를 선택해 줍니디ㅏ. 뒤에서 다시 말씀을 드리겠지만, 라마3는... ', 'bloggername': '초튜터 X 오씨네학교', 'bloggerlink': 'blog.naver.com/ocmarketing', 'postdate': '20240423'}, {'title': '라마(<b>Llama</b>) <b>3</b>.1 어떤 점이 달라졌을까? 성능부터 설치까지', 'link': 'https://blog.naver.com/spartacoding/223690422801', 'descripti